# Modeling
In this notebook, we will work through multiple iterations of our movie recommender system model. First, we will create a simpler model using just the movie reviews from IMDb. This will give us a baseline model that we can potentially improve on by adding more detailed features.